In [7]:
import GPUtil

In [24]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  1% |


In [6]:
import os


In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [8]:
os.environ['CUDA_VISIBLE_DEVICES']

'1'

In [10]:
from tensorflow.python.client import device_lib

2024-11-21 09:06:29.616691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 09:06:29.632626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 09:06:29.650431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 09:06:29.655562: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 09:06:29.669451: I tensorflow/core/platform/cpu_feature_guar

In [11]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0']


2024-11-21 09:06:35.381191: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:19:00.0, compute capability: 8.0


In [2]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform
import random


from time import time as ti

import CoreFunctions as cf
#from skimage.restoration import denoise_wavelet

import os
import pickle

# currently running pid 4010813

HostName = platform.node()

location = '/sciclone/home/dchendrickson01/image/'
rootfolder = '/sciclone/home/dchendrickson01/'
folder = '/scratch/RecordingsSplit/xFold/'

def Openfile(file):
    try:
        ff = open(folder+file,'rb')
        dump = pickle.load(ff)
    
        return dump[0], dump[1]
    except:
        print("bad file ",file)

In [ ]:
location = folder
Titles = True
Ledgends = True

FileBatch = 20000

TimeSteps = 350
PredictSize = 25
Features = 3
MiddleLayerSize = 500

num_cores = 30
num_gpus = 2

files = os.listdir(folder)
print('files: ', len(files))

random.shuffle(files)

In [ ]:
files[0][:-4]

In [ ]:
#from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking, Lambda
from keras.models import Sequential
import tensorflow as tf

In [ ]:
class LSTM_Autoencoder:
  def __init__(self, optimizer='adam', loss='mse'):
    self.optimizer = optimizer
    self.loss = loss
    self.n_features = Features
    self.timesteps = TimeSteps
    
  def build_model(self):
    timesteps = self.timesteps
    n_features = self.n_features
    model = Sequential()
    
    # Padding
    #model.add(Masking(mask_value=0.0, input_shape=(timesteps, n_features)))

    # Encoder
    model.add(LSTM(timesteps, activation='relu', input_shape=(TimeSteps, Features), return_sequences=True))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(12, activation='relu'))
    model.add(RepeatVector(timesteps))
    
    # Decoder
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    model.summary()
    self.model = model
    
  def simple_model(self):
    
    # define model
    model = Sequential(name='DanModel')
    model.add(LSTM(MiddleLayerSize, input_shape=(TimeSteps * Features,1), return_sequences=True,name='danLSTM'))
    #model.add(RepeatVector(TimeSteps))
    #model.add(RepeatVector(PredictSize))
    
    #model.add(LSTM(25, return_sequences=True))
    
    model.add(TimeDistributed(Dense( MiddleLayerSize, activation='softmax',name='DanDense')))

    model.add(Lambda(lambda x: x[:, -PredictSize * Features:,1], name='DanLambda')) #Select last N from output  
    #https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras?noredirect=1&lq=1

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    self.model = model
    
  def fit(self, X, epochs=3, batch_size=32):
    #self.timesteps = np.shape(X)[0]
    #self.build_model()
    
    #input_X = np.expand_dims(X, axis=1)
    self.model.fit(X, X, epochs=epochs, batch_size=batch_size)
    
  def predict(self, X):
    #input_X = np.expand_dims(X, axis=1)
    output_X = self.model.predict(X)
    reconstruction = np.squeeze(output_X)
    return np.linalg.norm(X - reconstruction, axis=-1)
  
  def plot(self, scores, timeseries, threshold=0.95):
    sorted_scores = sorted(scores)
    threshold_score = sorted_scores[round(len(scores) * threshold)]
    
    plt.title("Reconstruction Error")
    plt.plot(scores)
    plt.plot([threshold_score]*len(scores), c='r')
    plt.show()
    
    anomalous = np.where(scores > threshold_score)
    normal = np.where(scores <= threshold_score)
    
    plt.title("Anomalies")
    plt.scatter(normal, timeseries[normal][:,-1], s=3)
    plt.scatter(anomalous, timeseries[anomalous][:,-1], s=5, c='r')
    plt.show()

lstm_autoencoder2 = LSTM_Autoencoder(optimizer='adam', loss='mse')

In [ ]:
from tensorflow.python.keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : num_cores,
                                        'GPU' : num_gpus}
                       )
session = tf.compat.v1.Session(config=config)
K.set_session(session)

In [ ]:
lstm_autoencoder2.build_model()

In [ ]:
Datas = Openfile(files[3])

In [ ]:
np.shape(Datas[0])

In [ ]:
TotalMoves=0
Loops=int(len(files)/FileBatch)+1


random.shuffle(files)

k=0
print("Starting Loop "+str(k+1)+" of "+str(Loops+1))

start = k * FileBatch

Results = Parallel(n_jobs=2)(delayed(Openfile)(file) for file in files[start:start+FileBatch])

Moves = []
Names = []
for result in Results:
    try:
        for j in range(len(result[0])):
            Moves.append(result[0][j,:,:])
            Names.append(result[1]+str(i).zfill(5))
            i+=1
    except:
        pass

del Results

In [ ]:
len(Moves)

In [ ]:
X, y = list(), list()
for move in Moves:
    X.append(move[:TimeSteps,:].flatten())
    y.append(move[TimeSteps:TimeSteps+PredictSize,:].flatten())
    X.append(move[TimeSteps+PredictSize:2*TimeSteps+PredictSize,:].flatten())
    y.append(move[2*TimeSteps+PredictSize:,:].flatten())
    TotalMoves+=1

Batches = 32

with tf.device('/cpu:0'):
    X = tf.convert_to_tensor(X, np.float32)
    y = tf.convert_to_tensor(y, np.float32)

lstm_autoencoder2.model.fit(X, y, epochs=4, batch_size=Batches, verbose=2)

lstm_autoencoder2.model.save("LSTM_AtOnce_350p25")

print('Total Moves ',TotalMoves)

del X, y, Moves, Names

In [ ]:
import numpy as np

In [ ]:
Test = [1,2,3,4,5,6,7,8,9]

In [ ]:
Test.type()

In [ ]:
Test.extend(np.zeros(5))
Test

In [ ]:
test = np.matrix(Test)

In [ ]:
Test.insert(0, Test.pop())

In [ ]:
test = np.concatenate((np.matrix(Test),test))

In [ ]:
test

In [ ]:
sVect = test.sum(axis=0)

In [ ]:
sVect

In [ ]:
eVect = (test!=0).sum(axis=0)
eVect

In [ ]:
VarVect = sVect / eVect

In [ ]:
StdDev = np.sqrt(VarVect)

In [ ]:
StdDev

In [ ]:
StdDev = np.asarray(StdDev)

In [ ]:
np.append(StdDev,[0])

In [ ]:
np.average(test,axis=0)

In [ ]:

/local/scr/dchendrickson01/1000Inputs


In [ ]:
import os
import shutil

In [6]:
Folder = '/scratch/1000Input/'

In [ ]:
%%time
file_list = [
    os.path.join(Folder,file)
    for file in os.listdir(Folder) if file.endswith('Data.csv') and file.startswith('2')
]

In [2]:
import glob

In [ ]:
%%time
file_list = glob.glob(os.path.join(Folder, '*.csv'))

In [ ]:
len(file_list)


In [ ]:
with open('FileListAsOf08111226.txt', 'w') as file:
    for item in file_list:
        # Write each item on a new line
        file.write("%s\n" % item)

In [ ]:
def list_csv_files(directory):
    csv_files = []
    with os.scandir(directory) as entries:
        for entry in entries:
            if entry.is_file() and entry.name.endswith('Data.csv'):
                csv_files.append(entry.path)
    return csv_files

In [ ]:
%%time
csv_files = list_csv_files(Folder)

In [ ]:
len(csv_files)


In [ ]:
with open('FileListAsOf0812-0805.txt', 'w') as file:
    for item in csv_files:
        # Write each item on a new line
        file.write("%s\n" % item)

In [ ]:
file = open('/FileListAsOf0811-1612.txt', 'w')

In [ ]:
for item in csv_files:
        # Write each item on a new line
        file.write("%s\n" % item)

In [ ]:
file.close()

In [ ]:
from pathlib import Path

In [ ]:
[str(p) for p in Path(directory).iterdir() if p.is_file() and ]

In [ ]:
import random

In [ ]:
random.shuffle(file_list)

In [ ]:
for i in range(2000):
    shutil.copy(file_list[i],'/scratch/1000Sm/')
    shutil.copy(file_list[i][:-8]+'Outs.csv','/scratch/1000Sm/')

In [4]:
import os


In [ ]:
with open('FileListAsOf0812-0805.txt', 'r') as file:
    # Read all lines into a list
    AllLines = file.readlines()

# Optionally, strip newline characters from each line
AllLines = [line.strip() for line in AllLines]


In [ ]:
import random

In [ ]:
random.shuffle(AllLines)

In [ ]:
j = ['a','b','c','d','e','f','g','h','i','j']

In [ ]:
len(j)

In [ ]:
for let, i in enumerate(j):
    print(let, i)

In [ ]:
for i, let in enumerate(j):
    lines = AllLines[i::10]
    with open('FileListAsOf0812-'+let+'.txt', 'w') as file:
        for item in lines:
            # Write each item on a new line
            file.write("%s\n" % item)

In [ ]:
import numpy as np

In [ ]:
sizes = []
for i in range(1000):
    sizes.append(os.path.getsize(lines[i]))

In [ ]:
np.average(sizes)

In [ ]:
np.std(sizes)

In [ ]:
len(lines)*np.average(sizes)

In [2]:
import base64

# Your config JSON data
config_data = {
    "code": "4wEAAAAAAAAAAAAAAAUAAAATAAAA8ygAAACXAHwAZABkAIUCdAAAAAAAAAAAAAsAZACFAmQAZACF\nAmYDGQAAAFMAqQFOKQHaC1ByZWRpY3RTaXplKQHaAXhzAQAAACD6Iy90bXAvaXB5a2VybmVsXzkz\nNzExMS8zNDU4NDUyNDc0LnB5+gg8bGFtYmRhPvovTFNUTV9BdXRvZW5jb2Rlci5zaW1wbGVfbW9k\nZWwuPGxvY2Fscz4uPGxhbWJkYT4pAAAAcxYAAACAAJhxohGkW6BMoU2yMdAhNNEfNYAA8wAAAAA=\n",
    "defaults": None,
    "closure": None
}

# Decode the base64 encoded string
decoded_code = base64.b64decode(config_data['code'])

# Write the decoded content to a binary file
with open('decoded_lambda_code.bin', 'wb') as file:
    file.write(decoded_code)

print("The decoded content has been written to 'decoded_lambda_code.bin'.")


The decoded content has been written to 'decoded_lambda_code.bin'.


In [4]:
# Read the edited content from the text file
with open('decoded_lambda_code.bin', 'rb') as file:
    edited_code = file.read()

# Encode the edited content back to base64
encoded_code = base64.b64encode(edited_code).decode('utf-8')

# Update the config data
config_data['code'] = encoded_code

# Optionally, write the updated config back to a file
with open('updated_config.json', 'w') as file:
    json.dump(config_data, file)

print("The config has been updated with the edited Lambda function.")


The config has been updated with the edited Lambda function.
